    0 = neutral,                0 - нейтральність
    1 = anger,                  1 - злість
    2 = contempt,               2 - зневага
    3 = disgust,                3 - огида 
    4 = fear,                   4 - страх
    5 = happy,                  5 - радість 
    6 = sadness,                6 - сум 
    7 = surprise                7 - здивування 

In [28]:
import os
from shutil import copy, move

import numpy as np
import pandas as pd

In [5]:
emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]

if not os.listdir('sorted_data'):
    print('No emotions within sorted_data detected. Creating new')
    for em in emotions:
        os.makedirs('sorted_data/'+em)

In [78]:
encoded_emotions = {t: em for t, em in enumerate(emotions)}

In [79]:
encoded_emotions

{0: 'neutral',
 1: 'anger',
 2: 'contempt',
 3: 'disgust',
 4: 'fear',
 5: 'happy',
 6: 'sadness',
 7: 'surprise'}

Get all participants of the dataset

In [90]:
img_path = 'src_images'
annotations_path = 'src_labels'
destination = 'sorted_data'

raw_collector = []

participants = os.listdir(img_path)
labels = os.listdir(annotations_path)
# Map emotions with folder names
emo_map = dict(zip(emotions, [f'{k:03}' for k in range(7)]))

for person in sorted(labels):
    for emotion in os.listdir(os.path.join(annotations_path, person)):
        emotion_path = os.path.join(annotations_path, person, emotion)
        if os.listdir(emotion_path):
            with open(os.path.join(emotion_path, os.listdir(emotion_path)[0]), 'r') as f:
                sample = [os.listdir(emotion_path)[0], f.read()]
                raw_collector.append(sample)


In [48]:
import re

In [63]:
collector

[[['S005', '001', '00000011', 'emotion.txt'], 3.0],
 [['S010', '004', '00000019', 'emotion.txt'], 1.0],
 [['S010', '002', '00000014', 'emotion.txt'], 7.0],
 [['S010', '006', '00000015', 'emotion.txt'], 5.0],
 [['S011', '005', '00000020', 'emotion.txt'], 3.0],
 [['S011', '004', '00000021', 'emotion.txt'], 1.0],
 [['S011', '002', '00000022', 'emotion.txt'], 6.0],
 [['S011', '001', '00000016', 'emotion.txt'], 7.0],
 [['S011', '006', '00000013', 'emotion.txt'], 5.0],
 [['S011', '003', '00000014', 'emotion.txt'], 4.0],
 [['S014', '005', '00000017', 'emotion.txt'], 5.0],
 [['S014', '002', '00000016', 'emotion.txt'], 6.0],
 [['S014', '001', '00000029', 'emotion.txt'], 7.0],
 [['S014', '003', '00000030', 'emotion.txt'], 1.0],
 [['S022', '005', '00000032', 'emotion.txt'], 1.0],
 [['S022', '001', '00000030', 'emotion.txt'], 7.0],
 [['S022', '006', '00000017', 'emotion.txt'], 3.0],
 [['S026', '002', '00000016', 'emotion.txt'], 6.0],
 [['S026', '001', '00000015', 'emotion.txt'], 7.0],
 [['S026', '

In [82]:
collector = []

for filename, emotion in raw_collector:
    file_metadata = filename.split('_')
    emotion = float(emotion.replace(' ', '').replace('e+00\n', ''))
    collector.append([file_metadata, emotion])

##### Create a backup

In [65]:
with open('backup_0.csv', 'w') as f:
    f.write(str(collector).replace('.0],', '\n').replace('[', '').replace(']', ''))

#### Move images to appropriate folders

In [91]:
for [person, collection, img, ll], label in collector:
    # Such a shit :)
    assert label > 0, f'{person, collection} has neutral ({label}) label'
    
    path_to_img_folder = os.path.join(img_path, person, collection)
    
    images = sorted(os.listdir(path_to_img_folder))
    # First 20% of images are neutral. The last (suppose 50%) are emotional.
    n = len(images)
    for i, image in enumerate(images, 1):
        img_with_path = os.path.join(path_to_img_folder, image)
        # Mark first 20% of images as neutral
        if i / n <= 0.2:
            copy(img_with_path, os.path.join(destination, encoded_emotions[0], image))
        elif i / n >= 0.5:
            copy(img_with_path, os.path.join(destination, encoded_emotions[int(label)], image))
        # Mark last 50% of images as not neutral
        #print(img_with_path, label, encoded_emotions[int(label)], round(100 * i / n, 2))
    

Now go to folders & throw out some images manually (especially from the neutral folder)